In [3]:
print('hello class')

hello class


In [1]:
%pip install -qU langchain langchain-openai langchain_chroma langchain_community langchainhub beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [15]:
import os

from dotenv import load_dotenv

load_dotenv()  # take environment variables

token = os.getenv("MY_TOKEN")
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1-nano"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=model, base_url=endpoint, api_key=token)

In [2]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [17]:
# Load, chunk and index the contents of the blog.
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2017-06-21-overview/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
splits = text_splitter.split_documents(docs)

# print(splits)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(
    model="text-embedding-3-small",
    base_url="https://models.inference.ai.azure.com",
    api_key=token,
))

In [1]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

print(prompt)

NameError: name 'vectorstore' is not defined

In [19]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [20]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [21]:
rag_chain.invoke("What is Convolutional Neural Networks?")


'Convolutional Neural Networks (CNNs) are a type of feed-forward artificial neural network inspired by the organization of the visual cortex. They use convolutional layers with kernels or filters that slide across input images to detect features like edges, shapes, and textures. CNNs are widely used for image recognition and processing tasks.'